![erd](./ERD.png)

## Instruction

1. ใช้ project myshop เดิมจากของ WEEK 4 มาทำต่อได้เลยครับ (ใครที่หา project ของ WEEK 4 ไม่เจอแล้ว ให้กลับไปทำตามขั้ยตอนใน week4_exercises.ipynb)
2. ใช้ database `shop` เดิมได้เลยเช่นกันครับ

In [5]:
# import modules + set env
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ['DJANGO_SETTINGS_MODULE'] = 'myshop.settings'  
django.setup()


# ใช้ as เพื่อเปลี่ยนชื่อ function
import json
from shop.models import *
from django.db.models import F, Value as V, Avg, Sum, Count, Q
from django.db.models.functions import Concat
from django.utils import timezone
from datetime import datetime
from decimal import Decimal

### 1. annotate(), F()

1.1 ให้นักศึกษาค้นหาข้อมูล `Payment` โดยให้เพิ่ม field ราคาที่ลบกับส่วนลดแล้ว โดยกำหนดให้ชื่อ field ว่า "after_discount_price" โดยใช้แสดงข้อมูล 10 ตัวแรกเรียงตาม "after_discount_price" จากมากไปน้อย (0.25 คะแนน)

**หมายเหตุ: จะต้องใช้ annotate() นะครับ**

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
ID: 77, PRICE: 379000.00, DISCOUNT 19397.00, AFTER_DISCOUNT 359603.00
ID: 127, PRICE: 320450.00, DISCOUNT 14578.90, AFTER_DISCOUNT 305871.10
ID: 125, PRICE: 320399.00, DISCOUNT 17939.55, AFTER_DISCOUNT 302459.45
```

In [6]:
# annotate คือ method สำหรับการเพิ่มคอลัมน์ใหม่ในผลลัพธ์ของ QuerySet โดยการคำนวณจากข้อมูลที่มีอยู่ในฐานข้อมูล (ถ้าใช้ fliter จะ Error) (คล้ายการ การสร้างคอลัมน์คำนวณในคำสั่ง SQL เช่น SUM, AVG, COUNT, หรือการใช้ AS, GROUP BY)
# F() ใช้เพื่ออ้างถึงค่าของฟิลด์ของโมเดลใน query โดยตรงในระดับฐานข้อมูล โดยไม่ต้องดึงค่าของฟิลด์นั้นออก (คล้ายการ SELECT แล้วเอามา + - * / กัน)
# [:10] LIMIT 10 
# -after_discount_price คือ เรียง after_discount_price แบบ DESC (มากไปน้อย)

payment_discount = Payment.objects.annotate(
    after_discount_price = F("price") - F("discount")
    ).order_by('-after_discount_price')[:10]

# print(payment_discount.query) ดู query

for payment in payment_discount:
    print(f'ID: {payment.id}, PRICE: {payment.price}, DISCOUNT: {payment.discount}, AFTER_DISCOUNT: {payment.after_discount_price}')


ID: 92, PRICE: 1200500.00, DISCOUNT: 29433.25, AFTER_DISCOUNT: 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT: 46229.40, AFTER_DISCOUNT: 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT: 71407.25, AFTER_DISCOUNT: 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT: 105019.11, AFTER_DISCOUNT: 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT: 126859.95, AFTER_DISCOUNT: 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT: 113446.20, AFTER_DISCOUNT: 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT: 121922.64, AFTER_DISCOUNT: 1080267.36
ID: 77, PRICE: 379000.00, DISCOUNT: 19397.00, AFTER_DISCOUNT: 359603.00
ID: 127, PRICE: 320450.00, DISCOUNT: 14578.90, AFTER_DISCOUNT: 305871.10
ID: 125, PRICE: 320399.00, DISCOUNT: 17939.55, AFTER_DISCOUNT: 302459.45


1.2 ต่อเนื่องจากข้อ 1.1 ให้ filter เฉพาะข้อมูล `Payment` ที่มี "after_discount_price" มากกว่า 500,000 (0.25 คะแนน)

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
```

In [8]:
# annotate คือ method สำหรับการเพิ่มคอลัมน์ใหม่ในผลลัพธ์ของ QuerySet โดยการคำนวณจากข้อมูลที่มีอยู่ในฐานข้อมูล (ถ้าใช้ fliter จะ Error) 
# F() ใช้เพื่ออ้างถึงค่าของฟิลด์ของโมเดลใน query โดยตรงในระดับฐานข้อมูล โดยไม่ต้องดึงค่าของฟิลด์นั้นออก (คล้ายการ SELECT แล้วเอามา + - * / And Or Concat กัน)
#  __lte -> Less than or equal
# __gte -> Greater than or equal
# __lt -> Less than
#__gt -> Greater than
# [:10] LIMIT 10 
# -after_discount_price คือ เรียง after_discount_price แบบ DESC (มากไปน้อย)

payment_discount = Payment.objects.annotate(
    after_discount_price = F("price") - F("discount")
    ).filter(after_discount_price__gt=500000).order_by(F('after_discount_price').desc())[:10]

# print(payment_discount.query) ดู query (AS "after_discount_price" FROM "shop_payment" WHERE ("shop_payment"."price" - "shop_payment"."discount") > 500000)


for payment in payment_discount:
    print(f'ID: {payment.id}, PRICE: {payment.price}, DISCOUNT: {payment.discount}, AFTER_DISCOUNT: {payment.after_discount_price}')

ID: 92, PRICE: 1200500.00, DISCOUNT: 29433.25, AFTER_DISCOUNT: 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT: 46229.40, AFTER_DISCOUNT: 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT: 71407.25, AFTER_DISCOUNT: 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT: 105019.11, AFTER_DISCOUNT: 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT: 126859.95, AFTER_DISCOUNT: 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT: 113446.20, AFTER_DISCOUNT: 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT: 121922.64, AFTER_DISCOUNT: 1080267.36


1.3 ให้นักศึกษาเรียงลำดับข้อมูลลูกค้า (`Customer`) โดยเรียงลำดับตามลำดับตัวอักษร `น้อยไปมาก` จากชื่อเต็มของลูกค้า (`full_name`) โดยแสดง 5 คนแรก (0.5 คะแนน)

**Hint:** Field `full_name` นั้นจะต้องถูก annotate ขึ้นมาโดยการนำ `first_name` มาต่อกับ `last_name` โดยใช้ `Concat(*expressions, **extra)` 

**Hint:** แปลง object เป็น dict ใช้ `values()` [doc](https://docs.djangoproject.com/en/5.0/ref/models/querysets/#values)

```python
>>> Blog.objects.filter(name__startswith="Beatles").values()
<QuerySet [{'id': 1, 'name': 'Beatles Blog', 'tagline': 'All the latest Beatles news.'}]>
```

**Hint:** อยาก print dictionary สวยๆ ใช้ `json.dumps`

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

[doc](https://docs.djangoproject.com/en/5.0/ref/models/database-functions/#concat)

ตัวอย่าง Output 

```python
{
    "id": 17,
    "email": "anantaya.deena@gmail.com",
    "address": {
        "district": "Yan Nawa",
        "location": "60 Thanon Chan Road",
        "province": "Bangkok",
        "postal_code": 10120
    },
    "full_name": "Anantaya Tontong"
}
{
    "id": 25,
    "email": "bancha.zeyou@gmail.com",
    "address": {
        "district": "Watthana",
        "location": "6 Thong Lo Road",
        "province": "Bangkok",
        "postal_code": 10110
    },
    "full_name": "Bancha Kittisompong"
}
{
    "id": 19,
    "email": "chayapol.231@gmail.com",
    "address": {
        "district": "Hang Chat",
        "location": "160 Lampang Road",
        "province": "Lampang",
        "postal_code": 52190
    },
    "full_name": "Chayapol Komprach"
}
{
    "id": 4,
    "email": "dejwit.tt@gmail.com",
    "address": {
        "district": "Chiang Khan",
        "location": "150 Loei Road",
        "province": "Loei",
        "postal_code": 42110
    },
    "full_name": "Dejwit Tangjareonsakul"
}
{
    "id": 11,
    "email": "jack.maa@gmail.com",
    "address": {
        "district": "Bang Khen",
        "location": "88 Phahonyothin Road",
        "province": "Bangkok",
        "postal_code": 10220
    },
    "full_name": "Jack Maa"
}

```

In [12]:
# annotate คล้ายการ field ใหม่แต่ไม่ลง database
# ใช้ .value เพื่อแปลงผลลัพธ์ QuerySet เป็น dictionary แทนการคืนค่าเป็น instance (object) 

customers = Customer.objects.annotate( full_name = 
                                      Concat( F("first_name") , V(" "), F("last_name")) 
                                      ).order_by("full_name").values('id', 'email', 'address' , 'full_name')[:5]

# print(customers.query) ดู query
customer = list(customers)
print(json.dumps(customer, indent=4, sort_keys=False))

[
    {
        "id": 17,
        "email": "anantaya.deena@gmail.com",
        "address": {
            "district": "Yan Nawa",
            "location": "60 Thanon Chan Road",
            "province": "Bangkok",
            "postal_code": 10120
        },
        "full_name": "Anantaya Tontong"
    },
    {
        "id": 25,
        "email": "bancha.zeyou@gmail.com",
        "address": {
            "district": "Watthana",
            "location": "6 Thong Lo Road",
            "province": "Bangkok",
            "postal_code": 10110
        },
        "full_name": "Bancha Kittisompong"
    },
    {
        "id": 19,
        "email": "chayapol.231@gmail.com",
        "address": {
            "district": "Hang Chat",
            "location": "160 Lampang Road",
            "province": "Lampang",
            "postal_code": 52190
        },
        "full_name": "Chayapol Komprach"
    },
    {
        "id": 4,
        "email": "dejwit.tt@gmail.com",
        "address": {
            "district":

# FULL OUTPUT :
```JSON
[
    {
        "id": 17,
        "email": "anantaya.deena@gmail.com",
        "address": {
            "district": "Yan Nawa",
            "location": "60 Thanon Chan Road",
            "province": "Bangkok",
            "postal_code": 10120
        },
        "full_name": "Anantaya Tontong"
    },
    {
        "id": 25,
        "email": "bancha.zeyou@gmail.com",
        "address": {
            "district": "Watthana",
            "location": "6 Thong Lo Road",
            "province": "Bangkok",
            "postal_code": 10110
        },
        "full_name": "Bancha Kittisompong"
    },
    {
        "id": 19,
        "email": "chayapol.231@gmail.com",
        "address": {
            "district": "Hang Chat",
            "location": "160 Lampang Road",
            "province": "Lampang",
            "postal_code": 52190
        },
        "full_name": "Chayapol Komprach"
    },
    {
        "id": 4,
        "email": "dejwit.tt@gmail.com",
        "address": {
            "district": "Chiang Khan",
            "location": "150 Loei Road",
            "province": "Loei",
            "postal_code": 42110
        },
        "full_name": "Dejwit Tangjareonsakul"
    },
    {
        "id": 11,
        "email": "jack.maa@gmail.com",
        "address": {
            "district": "Bang Khen",
            "location": "88 Phahonyothin Road",
            "province": "Bangkok",
            "postal_code": 10220
        },
        "full_name": "Jack Maa"
    }
]
```

### 3. aggregation - count(), sum(), AVG()


3.1 ให้นักศึกษาหาค่าเฉลี่ยของราคาสินค้า (`Product.price`) ที่มีจำนวนคงเหลือ (`Product.remaining_amount`) มากกว่า 0 (0.25 คะแนน)

``` PYTHON
{'avg': Decimal('29308.000000000000')}
```


In [14]:
# aggregate คือ การคำนวณค่าที่ต้องการสำหรับทั้ง QuerySet และคืนค่าผลลัพธ์เป็น dictionary โดยไม่มีการแบ่งกลุ่มข้อมูล
product_price_avg = Product.objects.filter(remaining_amount__gt = 0).aggregate(avg = Avg("price"))

print(product_price_avg)

{'avg': Decimal('29308.000000000000')}


3.2 จงหาผลรวมราคา (`CartItem.product.price`) ที่อยู่ในตระกร้าสินค้าของเดือน `พฤษภาคม` (ดูจาก `Cart.create_date`) (0.5 คะแนน)

```PYTHON
{'sum': Decimal('9912555.00')}

```

In [14]:
# cart_may = Cart.objects.filter(create_date__month = 5)

# cart คือ cart_id (FK) ของตาราง CartItem ที่เชื่อมกับตาราง Cart แบบ 1-M
cart_sum_may = CartItem.objects.filter(cart__create_date__month= 5).aggregate(
    # product__price คือ การเรียกตาราง product โดยเรียก field price ในตาราง product ออกมา
    # สามารถ join ใน query ได้โดยใช้ __
    sum=Sum('product__price')
)

print(cart_sum_may)

{'sum': Decimal('9912555.00')}


3.3 ให้นักศึกษานับจำนวนสินค้าที่อยู่ประเภท `Electronics`,  `Jewelry` และ ราคาของสินค้าอยู่ในช่วง 8,000.00 - 50,000.00 (0.25 คะแนน)

```
PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1
```

In [16]:
product_category = ProductCategory.objects.filter(name__in=('Electronics', 'Jewelry'))

# ใช้ value เพื่อดึง field name ของตาราง ProductCatagory ออกมา
# Count นี้นับตาม
# โดยการใช้ การใช้ values() และ annotate() จะคล้ายการ GROUP BY ใน SQL (ใช้สำหรับจัดกลุ่มแถวของผลลัพธ์ที่มีค่าเหมือนกัน)

product_count_category = Product.objects.filter(
    Q( price__range = (8000, 50000) ) & Q(categories__in=product_category)
    ).values('categories__name').annotate(
        product_count=Count('id')
    )

# print(product_count_catagory.query) ดู query

# ต้องใช้ ['categories__name'] เพราะ ดึงฟิลด์จาก values ซึ่งเป็น list
for product in product_count_category:
    print(f"PRODUCT CATEGORY NAME: {product['categories__name']}, PRODUCT COUNT: {product['product_count']}")


PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1


In [17]:
product_electronics = ProductCategory.objects.get(name='Electronics')
product_jewerly = ProductCategory.objects.get(name='Jewelry')
# ใช้ value เพื่อดึง field name ของตาราง ProductCatagory ออกมา
# Count นี้นับตาม
# โดยการใช้ การใช้ values() และ annotate() จะคล้ายการ GROUP BY ใน SQL (ใช้สำหรับจัดกลุ่มแถวของผลลัพธ์ที่มีค่าเหมือนกัน)

product_count_category = Product.objects.filter(

    Q( price__range = (8000, 50000) ) , 
    (
        Q(categories=product_electronics) | Q(categories=product_jewerly)
        )
    )

# print(product_count_catagory.query) ดู query

# ต้องใช้ ['categories__name'] เพราะ ดึงฟิลด์จากความสัมพันธ์แบบ Many-to-Many 
print(f"PRODUCT CATEGORY NAME: {product_electronics.name}, PRODUCT COUNT: {product_count_category.filter(categories=product_electronics).count()}")
print(f"PRODUCT CATEGORY NAME: {product_jewerly.name}, PRODUCT COUNT: {product_count_category.filter(categories=product_jewerly).count()}")

ProductCategory object (2)
PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1


### 4. one-to-one & one-to-many

4.1 ให้นักศึกษาทำการ INSERT ข้อมูลใบสั่งซื้อ (`Order`) และการชำระเงิน (`Payment`) ของลูกค้าชื่อ `Manit Senapan` ตามรายการดังนี้ให้สมบูรณ์ (0.5 คะแนน)

**Hint:** ใน model `Payment` เรามีการเก็บค่า `price` เป็น Decimal ทำให้ค่าของราคาเป็น Decimal เช่นกัน)
[Decimal](https://docs.python.org/3/library/decimal.html)

- ออกใบสั่งซื้อวันที่ 5 สิงหาคม 2024
- ชำระเงินวันที่ 6 สิงหาคม 2024
- หมายเหตุ: `ฉันรวย อยากใช้เงินเยอะๆ`
    
    โดย Manit สั่งสิ้นค้าดั่งนี้

        - Diamond Stud Earrings จำนวน 1 ชิ้น

        - Sofa จำนวน 2 ชิ้น

        - Rose Gold Hoop Earrings จำนวน 1 ชิ้น
    
- โดยที่ Manit ได้รับส่วนลดชิ้นละ 10 % ของสินค้า และมีการระบุหมายเหตุตอนชำระเงินว่า `ลูกค้า VIP ของเรา`
    
- พร้อมชำระเงินโดยให้ 50 % ของราคาทั้งหมดชำระด้วยการแสกน QR code และที่เหลือชำระผ่านบัตรเคดิต

- จากนั้นให้ระบบแสดงผลการสร้างใบสั่งซื้อ และการชำระเงินของ Manit ให้ถูกต้อง


ตัวอย่าง output ที่ต้องการ
```PYTHON
{
    'order_id': 186,
    'order_date': '2024-08-05',
    'order_remark': 'ฉันรวย อยากใช้เงินเยอะๆ',
    'products': [
        {
            'product': 'Diamond Stud Earrings',
            'amount': 1,
            'price': 320000.0,
            'discount': 32000.0
        },
        {
            'product': 'Sofa', 
            'amount': 2, 
            'price': 14000.0, 
            'discount': 1400.0
        },
        {
            'product': 'Rose Gold Hoop Earrings',
            'amount': 1,
            'price': 1200000.0,
            'discount': 120000.0
        }
    ],
    'payment_date': '2024-08-06',
    'payment_remark': 'ลูกค้า VIP ของเรา',
    'payment_method': [
        {
            'method': 'QR', 
            'price': 767000.0
        },
        {
            'method': 'CREDIT', 
            'price': 767000.0
        }
    ]
}

```

In [18]:
# code here - INSERT (วิธี 1 No loop)

# ตรวจสอบและดึงลูกค้า
customer = Customer.objects.get( 
    first_name='Manit',
    last_name='Senapan'
)

# ตรวจสอบและดึงสินค้า
diamond = Product.objects.get(name='Diamond Stud Earrings')
sofa = Product.objects.get(name='Sofa')
rose_gold = Product.objects.get(name='Rose Gold Hoop Earrings')


# สร้าง Order
order = Order.objects.create(
    customer=customer,
    order_date='2024-08-05',
    remark='ฉันรวย อยากใช้เงินเยอะๆ'
)

# สร้าง Order_Item
# order=order คือ การเชื่อมโยง OrderItem กับ Order (ในโมเดลที่มีความสัมพันธ์แบบ One-to-Many หรือ Many-to-One)
order_item1 = OrderItem.objects.create(order=order, product=diamond, amount=1)
order_item2 = OrderItem.objects.create(order=order, product=sofa, amount=2)
order_item3 = OrderItem.objects.create(order=order, product=rose_gold, amount=1)


# คำนวนราคา
# diamond_earrings.price เป็นการเข้าถึง Field price ของ Obj. diamond 
# Decimal('0.10') เพื่อสร้างค่าทศนิยมที่มีความแม่นยำสูง คือสร้าง 0.10 นั้นเอง
total_price = (
    diamond.price * 1 + 
    sofa.price * 2 + 
    rose_gold.price * 1
)
total_discount = total_price * Decimal('0.10') # Manit ได้รับส่วนลดชิ้นละ 10 % ของสินค้า
final_price = total_price - total_discount

# สร้าง Payment
payment = Payment.objects.create(
    order=order,
    payment_date=datetime(2024, 8, 5),
    remark='ลูกค้า VIP ของเรา',
    price=final_price,
    discount=total_discount
)

# สร้าง PaymentMethods
payment_method1 = PaymentMethod.objects.create(
    payment=payment,
    method='QR',
    price=total_price / 2 # พร้อมชำระเงินโดยให้ 50 % ของราคาทั้งหมดชำระด้วยการแสกน QR code และที่เหลือชำระผ่านบัตรเคดิต  ( ใช้ Decimal('0.50') ก็ได้ )
)
payment_method2 = PaymentMethod.objects.create(
    payment=payment,
    method='CREDIT',
    price=total_price / 2 # พร้อมชำระเงินโดยให้ 50 % ของราคาทั้งหมดชำระด้วยการแสกน QR code และที่เหลือชำระผ่านบัตรเคดิต  ( ใช้ Decimal('0.50') ก็ได้ )
)

# สร้าง PaymentItems (ลอง) + ให้ดึงข้อมูลมาแสดงผลง่ายขึ้น
payment_item1 = PaymentItem.objects.create(
    payment=payment,
    order_item=order_item1,
    price=diamond.price * 1,
    discount=diamond.price * 1 * Decimal('0.10')
)
payment_item2 = PaymentItem.objects.create(
    payment=payment,
    order_item=order_item2,
    price=sofa.price * 2,
    discount=sofa.price * 2 * Decimal('0.10')
)
payment_item3 = PaymentItem.objects.create(
    payment=payment,
    order_item=order_item3,
    price=rose_gold.price * 1,
    discount=rose_gold.price * 1 * Decimal('0.10')
)


In [9]:
# code here - INSERT (วิธี 2 loop)

# ตรวจสอบและดึงลูกค้า
customer = Customer.objects.get( 
    first_name='Manit',
    last_name='Senapan'
)

# ตรวจสอบและดึงสินค้า
diamond = Product.objects.get(name='Diamond Stud Earrings')
sofa = Product.objects.get(name='Sofa')
rose_gold = Product.objects.get(name='Rose Gold Hoop Earrings')


# สร้าง Order
order = Order.objects.create(
    customer=customer,
    order_date='2024-08-05',
    remark='ฉันรวย อยากใช้เงินเยอะๆ'
)

# เปลี่ยนให้เป็น list เพื่อให้ใช้ loop เข้าถึงได้
# สร้าง Order_Item
# order=order คือ การเชื่อมโยง OrderItem กับ Order (ในโมเดลที่มีความสัมพันธ์แบบ One-to-Many หรือ Many-to-One)
order_items = [
    OrderItem.objects.create(order=order, product=diamond, amount=1),
    OrderItem.objects.create(order=order, product=sofa, amount=2),
    OrderItem.objects.create(order=order, product=rose_gold, amount=1)
]

# เปลี่ยนมาใช้ loop เข้าถึง
# คำนวนราคา
# diamond_earrings.price เป็นการเข้าถึง Field price ของ Obj. diamond 
# Decimal('0.10') เพื่อสร้างค่าทศนิยมที่มีความแม่นยำสูง คือสร้าง 0.10 นั้นเอง
total_price = sum(item.product.price * item.amount for item in order_items) # เอา ราคา * จำนวน โดยเข้า loop ผ่าน order_items
total_discount = sum(item.product.price * item.amount * Decimal('0.10') for item in order_items) # Manit ได้รับส่วนลดชิ้นละ 10 % ของสินค้า
final_price = total_price - total_discount

# สร้าง Payment
payment = Payment.objects.create(
    order=order,
    payment_date=datetime(2024, 8, 5),
    remark='ลูกค้า VIP ของเรา',
    price=final_price,
    discount=total_discount
)

# สร้าง PaymentMethods
payment_method1 = PaymentMethod.objects.create(
    payment=payment,
    method='QR',
    price=total_price / 2 # พร้อมชำระเงินโดยให้ 50 % ของราคาทั้งหมดชำระด้วยการแสกน QR code และที่เหลือชำระผ่านบัตรเคดิต  ( ใช้ Decimal('0.50') ก็ได้ )
)
payment_method2 = PaymentMethod.objects.create(
    payment=payment,
    method='CREDIT',
    price=total_price / 2 # พร้อมชำระเงินโดยให้ 50 % ของราคาทั้งหมดชำระด้วยการแสกน QR code และที่เหลือชำระผ่านบัตรเคดิต  ( ใช้ Decimal('0.50') ก็ได้ )
)

# สร้าง PaymentItems (ลอง) + ให้ดึงข้อมูลมาแสดงผลง่ายขึ้น
for item in order_items:
    PaymentItem.objects.create(
        payment=payment,
        order_item=item,
        price=item.product.price * item.amount,
        discount=item.product.price * item.amount * Decimal('0.10')
    )


dict = {
    'order_id': order.id,
    'order_date': str(order.order_date),
    'order_remark': order.remark,
    'products': [
        {
            'product': item.product.name,
            'amount': item.amount,
            'price': float(item.product.price * item.amount),
            'discount': float(item.product.price * item.amount * Decimal('0.10'))
        }
        for item in order_items
    ],
    'payment_date': str(payment.payment_date),
    'payment_remark': payment.remark,
    'payment_method': [
        {
            'method': method.method,
            'price': float(method.price)
        }
        for method in payment.paymentmethod_set.all()
    ]
}

json_output = json.dumps(dict, ensure_ascii=False, indent=4)

print(json_output)

{
    "order_id": 194,
    "order_date": "2024-08-05",
    "order_remark": "ฉันรวย อยากใช้เงินเยอะๆ",
    "products": [
        {
            "product": "Diamond Stud Earrings",
            "amount": 1,
            "price": 320000.0,
            "discount": 32000.0
        },
        {
            "product": "Sofa",
            "amount": 2,
            "price": 14000.0,
            "discount": 1400.0
        },
        {
            "product": "Rose Gold Hoop Earrings",
            "amount": 1,
            "price": 1200000.0,
            "discount": 120000.0
        }
    ],
    "payment_date": "2024-08-05 00:00:00",
    "payment_remark": "ลูกค้า VIP ของเรา",
    "payment_method": [
        {
            "method": "QR",
            "price": 767000.0
        },
        {
            "method": "CREDIT",
            "price": 767000.0
        }
    ]
}


In [19]:
# code here - แสดงผล - 1

# ดึงข้อมูล PaymentItem
payment_items = PaymentItem.objects.filter(pk__range=(399, 401))

# Data Format JSON
data = {
    'order_id': None,
    'order_date': None,
    'order_remark': None,
    'products': [],
    'payment_date': None,
    'payment_remark': None,
    'payment_method': []
}


# ถ้ามีข้อมูลใน queryset ให้ดำเนินการต่อ
if payment_items:
    
    # ดึงข้อมูลการสั่งซื้อจาก PaymentItem 
    # payment_items[0] หมายถึงการดึงรายการแรกจาก queryset ของ PaymentItem ที่มี pk อยู่ในช่วง 369 ถึง 374
    first_item = payment_items[0]
    order = first_item.order_item.order
    payment = first_item.payment
    # เอาข้อมูลเข้า JSON Fomat Data
    data['order_id'] = order.id
    data['order_date'] = str(order.order_date)
    data['order_remark'] = order.remark

    # ดึงข้อมูลสินค้าและการลดราคา
    for item in payment_items:
        product_info = {
            'product': item.order_item.product.name,
            'amount': item.order_item.amount,
            'price': float(item.price),
            'discount': float(item.discount)
        }
        # เอาข้อมูลเข้า JSON Fomat Data
        data['products'].append(product_info)

    # ดึงข้อมูลการชำระเงิน
    data['payment_date'] = str(payment.payment_date)
    data['payment_remark'] = payment.remark

    # ดึงข้อมูล PaymentMethod
    # payment=payment คือ  payment = first_item.payment อยู่ใน payment_items = PaymentItem.objects.filter(pk__range=(384, 386)) แล้ว
    payment_methods = PaymentMethod.objects.filter(payment=payment)
    for method in payment_methods:
        method_info = {
            'method': method.method,
            'price': float(method.price)
        }

        # เอาข้อมูลเข้า JSON Fomat Data
        data['payment_method'].append(method_info)

    # แปลงข้อมูลเป็น JSON
    # ensure_ascii=False จะช่วยให้ข้อมูล JSON ที่มีอักขระภาษาไทย (กำหนดให้การเข้ารหัสเอาต์พุต JSON)
    json_output = json.dumps(data, ensure_ascii=False, indent=4)

    # แสดงผล JSON
    print(json_output)

else:
    print("No Data to query")



{
    "order_id": 195,
    "order_date": "2024-08-05",
    "order_remark": "ฉันรวย อยากใช้เงินเยอะๆ",
    "products": [
        {
            "product": "Diamond Stud Earrings",
            "amount": 1,
            "price": 320000.0,
            "discount": 32000.0
        },
        {
            "product": "Sofa",
            "amount": 2,
            "price": 14000.0,
            "discount": 1400.0
        },
        {
            "product": "Rose Gold Hoop Earrings",
            "amount": 1,
            "price": 1200000.0,
            "discount": 120000.0
        }
    ],
    "payment_date": "2024-08-05",
    "payment_remark": "ลูกค้า VIP ของเรา",
    "payment_method": [
        {
            "method": "QR",
            "price": 767000.0
        },
        {
            "method": "CREDIT",
            "price": 767000.0
        }
    ]
}


# FULL OUTPUT:
``` JSON
{
    "order_id": 195,
    "order_date": "2024-08-05",
    "order_remark": "ฉันรวย อยากใช้เงินเยอะๆ",
    "products": [
        {
            "product": "Diamond Stud Earrings",
            "amount": 1,
            "price": 320000.0,
            "discount": 32000.0
        },
        {
            "product": "Sofa",
            "amount": 2,
            "price": 14000.0,
            "discount": 1400.0
        },
        {
            "product": "Rose Gold Hoop Earrings",
            "amount": 1,
            "price": 1200000.0,
            "discount": 120000.0
        }
    ],
    "payment_date": "2024-08-05",
    "payment_remark": "ลูกค้า VIP ของเรา",
    "payment_method": [
        {
            "method": "QR",
            "price": 767000.0
        },
        {
            "method": "CREDIT",
            "price": 767000.0
        }
    ]
}

``` 

In [16]:
# ลบข้อมูลที่ TEST
# payment_items_to_delete = Order.objects.filter(pk__range=(186, 190)).delete()


### 5. many-to-many

5.1 ให้นักศึกษาค้นหาข้อมูลสินค้า (`Product`) ที่อยู่ในประเภทสินค้า "Information technology" 10 รายการแรก (เรียงลำดับด้วย `Product.id`) และแสดงชื่อประเภทสินค้า (`ProductCategory`) (0.25 คะแนน)

ตัวอย่าง Output บางส่วน

```
PRODUCT ID: 1, PRODUCT NAME: Smartphone, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 2, PRODUCT NAME: Laptop, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 3, PRODUCT NAME: Smart TV, PRODUCT CATEGORY: Information technology, Electronics
```

In [21]:
# Query ที่ตาราง Product ได้เลยเพราะ field categories นั้นอยู่ที่ ตาราง Product อยู่แล้ว 
products = Product.objects.filter(categories__name="Information Technology").order_by('id')[:10]


for product in products:
    # join เป็น String method ที่ใช้เพื่อเชื่อมต่อรายการของ String ให้เป็น String เดียว 
    # เชื่อมกับการที่ loop เอา product.categories ทั้งหมดออกมา
    # category.name คือการเข้าถึง Field name ของแต่ละ ProductCategory
    categories = ', '.join(category.name for category in product.categories.all())
    print(f"PRODUCT ID: {product.id}, PRODUCT NAME: {product.name}, PRODUCT CATEGORY: { categories }")

PRODUCT ID: 1, PRODUCT NAME: Smartphone, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 2, PRODUCT NAME: Laptop, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 3, PRODUCT NAME: Smart TV, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 4, PRODUCT NAME: Bluetooth Earphones, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 5, PRODUCT NAME: Tablet, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 6, PRODUCT NAME: Gaming Console, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 7, PRODUCT NAME: Digital Camera, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 8, PRODUCT NAME: Wireless Router, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 9, PRODUCT NAME: Portable Power Bank, PRODUCT CATEGORY: Information Technology, Electronics
PRODUCT ID: 10, PRODUCT NAME: Smartwatch, PRODUCT CATEGORY: Information Technology, Electronics


5.2 ให้นักศึกษาทำตามขั้นตอนดังนี้ (0.25 คะแนน)

    1. เปลี่ยนชื่อประเภทสินค้า `Books and Media` เป็น `Books and Toys` 
    2. ลบประเภท `Toys and Games` ออกโดยให้ใช้เป็น `Books and Toys` แทน
    3. ค้นหาว่าสินค้าที่มีประเภทสินค้าเป็น `Books and Toys` ทั้งหมดมีจำนวนเท่าไหร่

In [21]:
# 1.
books_and_media = ProductCategory.objects.get(pk=7)
books_and_media.name = 'Books and Toys'
books_and_media.save()
print(books_and_media.name)

toys_and_games = ProductCategory.objects.get(name='Toys and Games')
toys_and_games_products = Product.objects.filter(categories=toys_and_games)
for product in toys_and_games_products:
    product.categories.remove(toys_and_games)
    product.categories.add(books_and_media)

toys_and_games.delete()

product_count = len(Product.objects.filter(categories=books_and_media))
print(f"{product_count}")





Books and Toys
1


In [23]:
# 2. (1, {'shop.ProductCategory': 1}) 1 หมายถึงมีการลบวัตถุประเภท ProductCategory จำนวน 1 รายการ shop.ProductCategory: 1 ระบุว่ามีการลบวัตถุในโมเดล ProductCategory จำนวน 1 รายการ.


DoesNotExist: ProductCategory matching query does not exist.

In [24]:
# 3.


NameError: name 'books_and_media' is not defined

## เพิ่มเติม

1. `Q กับ F ต่างกับไง`
   
   ##### ANS `F()` ใช้เพื่ออ้างถึงค่าของฟิลด์ใน query โดยตรงในระดับฐานข้อมูลโดยไม่ต้องดึงค่าของฟิลด์นั้นออกมา 
   ##### `Q()` objects ใช้สำหรับสร้าง query ที่ซับซ้อนโดยใช้การเชื่อมโยงแบบตรรกะ (logical) เช่น AND, OR, NOT 

2. `annotate() และ aggregate() คือ ?`
    
   ##### ANS `aggregate()` คำนวณค่าที่ต้องการสำหรับทั้ง QuerySet และคืนค่าผลลัพธ์เป็น dictionary โดยไม่มีการแบ่งกลุ่มข้อมูล
   ##### `annotate()` ใช้เพื่อเพิ่มฟิลด์ที่คำนวณค่าที่ต้องการให้กับแต่ละแถวใน QuerySet โดยไม่ต้องลง Database จริง

3. `values() คือ ?`
   
   #### ANS `values()` คือ การแปลงผลลัพธ์ QuerySet เป็น dictionary แทนการคืนค่าเป็น instance (object) 
   #### ถ้าใช้ values() และ annotate() จะคล้ายการ GROUP BY ใน SQL (ใช้สำหรับจัดกลุ่มแถวของผลลัพธ์ที่มีค่าเหมือนกัน)

4. `join() คือ ?`
   
   #### ANS `join()` เป็นเมธอดของคลาส str ที่ใช้เพื่อเชื่อมต่อรายการของสตริงให้เป็นสตริงเดียว
   #### การใช้งาน
   ```` PYTHON
   separator.join(iterable)
   ````
   `separator` คือสตริงที่ต้องการใช้เป็นตัวเชื่อม
   `iterable` คือรายการของสตริงที่ต้องการรวม

   #### EX

   ```` PYTHON
   words = ["Hello", "world", "this", "is", "Python"]
   sentence = " ".join(words)
   print(sentence)  # Output: Hello world this is Python

   ````

5. `remove VS delete`
   
   #### ANS `remove` วิธีนี้มักใช้สำหรับความสัมพันธ์หลายต่อหลาย (many-to-many) โดยที่เราต้องการแค่ถอดความสัมพันธ์ออก แต่ไม่ลบวัตถุที่เกี่ยวข้องเอง (ลบข้อมูลในตาราง bridge entity ออกแต่ไม่ลบข้อมูลตัวเองออก)`delete` วิธีนี้ใช้สำหรับลบข้อมูลที่เป็นตัวอย่างของโมเดลออกจากฐานข้อมูลอย่างถาวร (ลบข้อมูลออกหมดเลย)


6. `1-M  Query ยังไง`
    
   #### ANS . กันไปได้เลยแต่ชื่อจะมี _set ต่อท้าย


   #### EX ตาราง
   ``` PYTHON 
   class Author(models.Model):
      name = models.CharField(max_length=100)
      age = models.IntegerField()


   class Publisher(models.Model):
      name = models.CharField(max_length=300)


   class Book(models.Model):
      name = models.CharField(max_length=300)
      pages = models.IntegerField()
      price = models.DecimalField(max_digits=10, decimal_places=2)
      rating = models.FloatField()
      authors = models.ManyToManyField(Author)
      publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE)
      pubdate = models.DateField()


   class Store(models.Model):
      name = models.CharField(max_length=300)
      books = models.ManyToManyField(Book)

   class Publisher(models.Model):
      name = models.CharField(max_length=300)

   class Book(models.Model):
      name = models.CharField(max_length=300)
      pages = models.IntegerField()
      price = models.DecimalField(max_digits=10, decimal_places=2)
      rating = models.FloatField()
      authors = models.ManyToManyField(Author)
      publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE)
      pubdate = models.DateField()

   class StoreContact(models.Model):
      mobile = models.CharField(max_length=20)
      email = models.EmailField(max_length=50, blank=True, null=True)
      address = models.TextField()
      store = models.OneToOneField(Store, on_delete=models.CASCADE)
   ```
      ### Get the publisher instance
      - penguin_pub = Publisher.objects.get(name="Penguin Books") 
      - books = penguin_pub.book_set.all()
      - ##### book_set คือฟิลด์ที่ Django สร้างขึ้นโดยอัตโนมัติเมื่อใช้ ForeignKey กับ related_name ที่ไม่ได้กำหนดในโมเดลของคุณ ฟิลด์นี้ช่วยให้คุณสามารถเข้าถึงชุดข้อมูลที่มีความสัมพันธ์แบบหนึ่งต่อหลาย (one-to-many) จากโมเดลที่มี ForeignKey (related_name สามารถกำหนดใน models ได้)
      - `Result = <QuerySet [<Book: Book object (2)>, <Book: Book object (3)>, <Book: Book object (4)>, <Book: Book object (5)>, <Book: Book object (6)>, <Book: Book object (7)>, <Book: Book object (8)>, <Book: Book object (9)>, <Book: Book object (10)>, <Book: Book object (11)>, <Book: Book object (12)>, <Book: Book object (13)>, <Book: Book object (14)>, <Book: Book object (15)>, <Book: Book object (16)>, <Book: Book object (17)>, <Book: Book object (18)>, <Book: Book object (19)>, <Book: Book object (20)>, <Book: Book object (21)>, '...(remaining elements truncated)...']>`
      ### How may books?
      - penguin_pub.book_set.count()
      -  `Result = 21`

7. `1-1  Query ยังไง`
   
   #### ANS . กันไปได้เลย !!!
   ### .store คือ ชื่อของ Relationship
   - contact.store
   - `Result = <Store: Store object (2)> `
   ### storecontact คือ ชื่อของ models
   - store.storecontact
   - `Result = <StoreContact: StoreContact object (1)>`
   ### สามารถ filter ได้คล้ายกับ one-to-many (storecontact คือการเชื่อมตาราง)
   - Store.objects.filter(storecontact__mobile="021113333")
   - `Result = <QuerySet [<Store: Store object (2)>]>`

8. `M-M  Query ยังไง`
   
   #### ANS . กันไปได้เลย !!!